<a href="https://colab.research.google.com/github/Carinaaa/ML-Learning-Path/blob/intro-LLM/OpenAI_for_compare_output_with_Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/ed-donner/llm_engineering


In [2]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from google.colab import userdata
import os


In [3]:
openai_key = userdata.get('OPENAI_API_KEY') # get the secret by the name
os.environ['OPENAI_API_KEY'] = openai_key # set it as an env var
openai = OpenAI() # create model using the env var

In [4]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
wiki_flat_coated_retriever = Website("https://en.wikipedia.org/wiki/Flat-coated_Retriever")

In [6]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
# And now let's build useful messages for GPT-4o-mini, using a function

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [9]:
# Time to bring it together: call the OpenAI API.

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [10]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [11]:
display_summary("https://en.wikipedia.org/wiki/Flat-coated_Retriever")

# Summary of the Flat-coated Retriever Wikipedia Page

The **Flat-coated Retriever** is a dog breed that originated in England, developed as a gun dog for retrieving both on land and in water. It is known for its friendly and playful temperament, often described as the "Peter Pan" of retrievers due to its youthful disposition.

## Description
- **Appearance**: 
  - Males stand 23-25 inches tall and weigh 60-80 lbs; females are slightly smaller.
  - The breed has a unique head shape, muscular jaws, and a glossy coat available in black and liver colors (yellow is disqualified in conformation shows).
  
- **Temperament**: 
  - Friendly, intelligent, and energetic, making them excellent companions and working dogs.
  - They can serve as drug-sniffers and are used in breeding programs for guide dogs.

## History
The breed gained popularity in the mid-19th century, partly due to its ancestry linked to St. John's water dog and Newfoundlands. It faced a decline in numbers post-World War II but has made a comeback since the 1960s, noted for conformation showing and as companion animals.

Notable achievements for the breed include wins at **Crufts**, with a highlighted victory for a Flat-coated Retriever named Jet in 2011 and another in 2022.

## Health
Recent studies show a life expectancy of approximately 11.7 years for Flat-coated Retrievers. Common health concerns include a high incidence of cancer-related deaths.

For further details, one can refer to the respective sections on appearance, temperament, history, and health within the article.